# Pre procesamiento

Como habíamos dicho es mejor pre procesar las palabras

In [1]:
from string import punctuation
import pandas as pd
import re
import unicodedata

Veamos el formato de los mensajes sin procesar

In [2]:
df = pd.read_csv('corpus.csv')
df.head(30)

,id,owner,date,hour,message
0,1,USER0,1/27/16,12:24,<Media omitted>
1,1,USER0,2/4/16,09:50,Lee Esta Historia! . Una mujer muy en...
2,1,USER0,4/4/16,11:28,Hola Dani!vos sabes a donde venden fotocopiado...
3,1,USER1,4/4/16,11:37,Le pregunto a mi jefe
4,1,USER0,4/4/16,11:38,Gracias!!!!Para cuando sabes?
5,1,USER0,4/4/16,11:38,💃
6,1,USER1,4/4/16,11:40,Le pregunto ahora a ver si sabe algo :P
7,1,USER0,4/4/16,11:59,👏👏👏👏👏
8,1,USER0,4/6/16,08:46,"✳Whatsapp le comunica, . que debido a la canti..."
9,1,USER1,4/6/16,08:46,Jajaja


Definamos las reglas de preprocesamiento

In [3]:
def rm_num(text):
    return re.sub('(\d+[\WxX]*[^ ]*)+', 'NUM', text)

def rm_tildes(text):
    return ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'))

def rm_media(text):
    return re.sub(r'<archivo omitido>|<media omitted>', 'MEDIA', text)

def rm_repeated(text):
    return re.sub(r'([aeourshmi])\1{2,}', r'\1', text)

def rm_ok(text):
    return re.sub('(ok)+[akis]*', 'ok', text)

def rm_ja(text):
    return re.sub('(ja[ja(js)]*)+', 'ja', text)

def rm_si(text):
    return re.sub('[s]+[i]+', 'si', text)

def clean_text(text):
    return rm_num(rm_si(rm_ok(rm_ja(rm_repeated(rm_media(rm_tildes(text.lower())))))))

## ¿Cómo tratar los signos de puntuación y los emojis?

Uno de los problemas a tratar es: ¿Qué hacer con los signos de puntuación? ¿Y con los emojis? ¿Se remueven o se tratan como palabra? ¿Algunas se remueven y otras se las tiene en cuenta como contexto? **¿Cómo tratar los emojis repetidos?**

Se deberá también tener en cuenta que muchas palabras aparecen concatenadas con los signos de puntuación y los emojis, ya sea por error de tipeo, o por el uso del signo. Ej. "hola.que haces??😄"

Definimos las reglas

In [17]:
from emoji import UNICODE_EMOJI

PUNCTUATION = punctuation + '¡' + '¿'
NOT_SIGNIFICANT = re.sub('[?!\"$]+', '', PUNCTUATION)
EMOJI = u''.join(c for c in UNICODE_EMOJI)

def rm_punct(text):
    """
    Remueve signos de puntuación. Reemplaza por un espacio.
    hola.que haces??😄 -> hola que haces  😄
    """
    return re.sub('{1}{0}{1}'.format(PUNCTUATION, '[]'), ' ', text)

def rm_extra_punct(text):
    """
    Remueve signos de puntuación repetidos y agrega espacio entre ellos.
    ???!!..' -> ? ! . '
    """
    return re.sub(r'({1}{0}{2})\1*'.format(PUNCTUATION, '[', ']'), r' \1 ', text)

def rm_not_signif(text):
    """
    Remueve signos de puntuación no significativos
    ???!!!..' -> ???!!!
    """
    return re.sub('{1}{0}{1}'.format(NOT_SIGNIFICANT, '[]'), ' ', text)

def rm_extra_emojis(text):
    """
    Remueve emojis repetidos y agrega espacio entre ellos.
    hola.que haces??😄😄😄 -> hola.que haces?? 😄
    """
    return re.sub(r'({1}{0}{2})\1*'.format(EMOJI, '[', ']'), r' \1 ', text)

def rm_emojis(text):
    """
    Remueve emojis
    hola.que haces??😄 -> hola.que haces??
    """
    return ''.join(c for c in text if c not in UNICODE_EMOJI)

def rm_extra_space(text):
    """
    Remueve espacios extra
    hola   que haces? ? -> hola que haces? ?
    """
    return ' '.join(text.split())

### Tratando los signos de puntuación y los emojis como palabra
En esta etapa se tratarán todos los signos y los emojis como palabras. El preprocesamiento consiste en añadir los espacios correspondientes y eliminar repeticiones de caracteres. Ej. "hola.que haces??😄" -> "hola . que haces ? 😄"

In [5]:
def pre1(text):
    return rm_extra_space(rm_extra_emojis(rm_extra_punct(clean_text(text))))

### Removiendo signos de puntuación menos significativos
Similar a la etapa anterior, pero removiendo sólo los signos que creemos no aportan al contexto. Ej. "hola.que haces??😄" -> "hola que haces ? 😄"

In [6]:
def pre2(text):
    return rm_extra_space(rm_extra_emojis(rm_extra_punct(rm_not_signif(clean_text(text)))))

### Quitando todos los signos de puntuación
En esta etapa removemos todos los signos de puntuación y preprocesamos los emojis. Ej. "hola.que haces??😄" -> "hola que haces 😄"

In [7]:
def pre3(text):
    return rm_extra_space(rm_extra_emojis(rm_punct(clean_text(text))))

### Quitando todos los signos de puntuación y los emojis
Para mejorar aún más la distribución, probamos quitando también los emojis.

In [18]:
def pre4(text):
    return rm_extra_space(rm_punct(rm_emojis(clean_text(text))))

Guardamos los mensajes preprocesados

In [9]:
df_cleaned_pre1 = pd.DataFrame(
    [
        (id, owner, date, hour, pre1(message))
        for (id, owner, date, hour, message) in [
            row for index, row in df.iterrows()
        ]
    ],
    columns=['id', 'owner', 'date', 'hour', 'message']
)

In [10]:
df_cleaned_pre2 = pd.DataFrame(
    [
        (id, owner, date, hour, pre2(message))
        for (id, owner, date, hour, message) in [
            row for index, row in df.iterrows()
        ]
    ],
    columns=['id', 'owner', 'date', 'hour', 'message']
)

In [11]:
df_cleaned_pre3 = pd.DataFrame(
    [
        (id, owner, date, hour, pre3(message))
        for (id, owner, date, hour, message) in [
            row for index, row in df.iterrows()
        ]
    ],
    columns=['id', 'owner', 'date', 'hour', 'message']
)

In [19]:
df_cleaned_pre4 = pd.DataFrame(
    [
        (id, owner, date, hour, pre4(message))
        for (id, owner, date, hour, message) in [
            row for index, row in df.iterrows()
        ]
    ],
    columns=['id', 'owner', 'date', 'hour', 'message']
)

In [13]:
df_cleaned_pre1.head(10)

,id,owner,date,hour,message
0,1,USER0,1/27/16,12:24,MEDIA
1,1,USER0,2/4/16,09:50,lee esta historia ! . una mujer muy enferma so...
2,1,USER0,4/4/16,11:28,hola dani ! vos sabes a donde venden fotocopia...
3,1,USER1,4/4/16,11:37,le pregunto a mi jefe
4,1,USER0,4/4/16,11:38,gracias ! para cuando sabes ?
5,1,USER0,4/4/16,11:38,💃
6,1,USER1,4/4/16,11:40,le pregunto ahora a ver si sabe algo : p
7,1,USER0,4/4/16,11:59,👏
8,1,USER0,4/6/16,08:46,"✳ whatsapp le comunica , . que debido a la can..."
9,1,USER1,4/6/16,08:46,ja


In [14]:
df_cleaned_pre2.head(10)

,id,owner,date,hour,message
0,1,USER0,1/27/16,12:24,MEDIA
1,1,USER0,2/4/16,09:50,lee esta historia ! una mujer muy enferma sono...
2,1,USER0,4/4/16,11:28,hola dani ! vos sabes a donde venden fotocopia...
3,1,USER1,4/4/16,11:37,le pregunto a mi jefe
4,1,USER0,4/4/16,11:38,gracias ! para cuando sabes ?
5,1,USER0,4/4/16,11:38,💃
6,1,USER1,4/4/16,11:40,le pregunto ahora a ver si sabe algo p
7,1,USER0,4/4/16,11:59,👏
8,1,USER0,4/6/16,08:46,✳ whatsapp le comunica que debido a la cantida...
9,1,USER1,4/6/16,08:46,ja


In [15]:
df_cleaned_pre3.head(10)

,id,owner,date,hour,message
0,1,USER0,1/27/16,12:24,MEDIA
1,1,USER0,2/4/16,09:50,lee esta historia una mujer muy enferma sono q...
2,1,USER0,4/4/16,11:28,hola dani vos sabes a donde venden fotocopiado...
3,1,USER1,4/4/16,11:37,le pregunto a mi jefe
4,1,USER0,4/4/16,11:38,gracias para cuando sabes
5,1,USER0,4/4/16,11:38,💃
6,1,USER1,4/4/16,11:40,le pregunto ahora a ver si sabe algo p
7,1,USER0,4/4/16,11:59,👏
8,1,USER0,4/6/16,08:46,✳ whatsapp le comunica que debido a la cantida...
9,1,USER1,4/6/16,08:46,ja


In [20]:
df_cleaned_pre4.head(10)

,id,owner,date,hour,message
0,1,USER0,1/27/16,12:24,MEDIA
1,1,USER0,2/4/16,09:50,lee esta historia una mujer muy enferma sono q...
2,1,USER0,4/4/16,11:28,hola dani vos sabes a donde venden fotocopiado...
3,1,USER1,4/4/16,11:37,le pregunto a mi jefe
4,1,USER0,4/4/16,11:38,gracias para cuando sabes
5,1,USER0,4/4/16,11:38,
6,1,USER1,4/4/16,11:40,le pregunto ahora a ver si sabe algo p
7,1,USER0,4/4/16,11:59,
8,1,USER0,4/6/16,08:46,whatsapp le comunica que debido a la cantidad ...
9,1,USER1,4/6/16,08:46,ja


In [21]:
df_cleaned_pre1.to_csv(path_or_buf='preprocessed_corpus_pre1.csv')
df_cleaned_pre2.to_csv(path_or_buf='preprocessed_corpus_pre2.csv')
df_cleaned_pre3.to_csv(path_or_buf='preprocessed_corpus_pre3.csv')
df_cleaned_pre4.to_csv(path_or_buf='preprocessed_corpus_pre4.csv')

## Distribución de palabras

Veamos como sería la nueva distribución

In [24]:
from collections import Counter
from itertools import chain

counter_pre1 = Counter(chain.from_iterable(df_cleaned_pre1.message.map(lambda x: x.split())))
counter_pre2 = Counter(chain.from_iterable(df_cleaned_pre2.message.map(lambda x: x.split())))
counter_pre3 = Counter(chain.from_iterable(df_cleaned_pre3.message.map(lambda x: x.split())))
counter_pre4 = Counter(chain.from_iterable(df_cleaned_pre4.message.map(lambda x: x.split())))

In [25]:
df_distribution_pre1 = pd.DataFrame(
    data={
        'word': list(counter_pre1.keys()),
        'counter': list(counter_pre1.values())
    }
)

In [26]:
df_distribution_pre2 = pd.DataFrame(
    data={
        'word': list(counter_pre2.keys()),
        'counter': list(counter_pre2.values())
    }
)

In [27]:
df_distribution_pre3 = pd.DataFrame(
    data={
        'word': list(counter_pre3.keys()),
        'counter': list(counter_pre3.values())
    }
)

In [28]:
df_distribution_pre4 = pd.DataFrame(
    data={
        'word': list(counter_pre4.keys()),
        'counter': list(counter_pre4.values())
    }
)

In [29]:
df_distribution_pre1.to_csv(path_or_buf='distribution_pre1.csv')
df_distribution_pre2.to_csv(path_or_buf='distribution_pre2.csv')
df_distribution_pre3.to_csv(path_or_buf='distribution_pre3.csv')
df_distribution_pre4.to_csv(path_or_buf='distribution_pre4.csv')

In [30]:
df_distribution_pre4.counter.median()

1.0

In [31]:
sum(df_distribution_pre4.counter == 1) / df_distribution_pre4.counter.count()

0.52885070834948111

Logramos reducir a un 53% la cantidad de palabras únicas

In [32]:
new_df = df_distribution_pre4[df_distribution_pre4.counter > 10]

In [33]:
new_df.counter.count()

5458

In [34]:
new_df.sort_values('counter', ascending=False).head(20)

,counter,word
9,29665,que
34,25772,de
13,22584,a
86,22479,no
20,21024,la
178,19970,ja
42,19880,el
28,18474,y
71,17302,NUM
0,16066,MEDIA


In [35]:
new_df.sort_values('counter').head(20)

,counter,word
19499,11,miembro
20874,11,sorprende
20932,11,spanish
4176,11,papo
13595,11,mendoza
16792,11,matecito
16710,11,soltera
16708,11,wasap
5862,11,vote
7794,11,desocupo


Las listas de palabras tienen más sentido